In [1]:
import cv2
import numpy as np
import cvzone
import math
import mediapipe as mp
import time
import code
caminput =0

In [2]:
##mediapipe hand detection
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
dir(code)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'cv2',
 'np',
 'skin_color_detection']

<img src="https://i.imgur.com/qpRACer.png "/>

<img src="https://camo.githubusercontent.com/7fbec98ddbc1dc4186852d1c29487efd7b1eb820c8b6ef34e113fcde40746be2/68747470733a2f2f6d65646961706970652e6465762f696d616765732f6d6f62696c652f706f73655f747261636b696e675f66756c6c5f626f64795f6c616e646d61726b732e706e67">

In [6]:
cap = cv2.VideoCapture(caminput)
StartCoords = []
pose = mp.solutions.pose.Pose(model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5)
hand = mp.solutions.hands.Hands(max_num_hands=2,
    min_detection_confidence=0.5)
prev_time = 0
new_time = 0
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image = np.zeros(frame.shape, dtype=np.uint8)
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        min_YCrCb = np.array([0,133,77],np.uint8)
        max_YCrCb = np.array([235,173,127],np.uint8)
        imageYCrCb = cv2.cvtColor(frame,cv2.COLOR_BGR2YCR_CB)
        skinRegionYCrCb = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)
        skinYCrCb = cv2.bitwise_and(frame, frame, mask = skinRegionYCrCb)



        img = cv2.flip(img, 1)
        img.flags.writeable = False
        res = holistic.process(img)
        pose_res = pose.process(img)
        hand_res = hand.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(img, res.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                  mp_drawing.DrawingSpec(color=(255, 110, 10), thickness=1, circle_radius=1),
                                    mp_drawing.DrawingSpec(color=(255, 256, 121), thickness=1, circle_radius=1)
                                    )
        # if hand_res.multi_hand_landmarks:
        #     for hand_landmarks in hand_res.multi_hand_landmarks:
        #         mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(img, pose_res.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                          mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                            )
        
        hand_limit = []

        if pose_res.pose_landmarks:
            left_hand_end_point = np.array([pose_res.pose_landmarks.landmark[14].x, pose_res.pose_landmarks.landmark[14].y, pose_res.pose_landmarks.landmark[14].z])
            left_hand_start_point = np.array([pose_res.pose_landmarks.landmark[16].x, pose_res.pose_landmarks.landmark[16].y, pose_res.pose_landmarks.landmark[16].z])
            por = 1/9
            Vect_left_hand = (left_hand_end_point - left_hand_start_point)*por + left_hand_start_point
            cv2.circle(img, (int(Vect_left_hand[0]*img.shape[1]), int(Vect_left_hand[1]*img.shape[0])), 5, (0, 0, 255), -1)
            right_hand_end_point = np.array([pose_res.pose_landmarks.landmark[13].x, pose_res.pose_landmarks.landmark[13].y, pose_res.pose_landmarks.landmark[13].z])
            right_hand_start_point = np.array([pose_res.pose_landmarks.landmark[15].x, pose_res.pose_landmarks.landmark[15].y, pose_res.pose_landmarks.landmark[15].z])
            Vect_right_hand = (right_hand_end_point - right_hand_start_point)*por + right_hand_start_point
            # cv2.circle(img, (int(Vect_right_hand[0]*img.shape[1]), int(Vect_right_hand[1]*img.shape[0])), 5, (0, 0, 255), -1)
            hand_limit.append([int(Vect_right_hand[0]*img.shape[1]), int(Vect_right_hand[1]*img.shape[0])])
        new_time = time.time()
        fps = 1/(new_time-prev_time)

        for i, (x, y) in enumerate(hand_limit):
            print(i, (x,y))
            part = img[
                y-10:y+10,
                x-10:x+10
            ]

            if len(part) <= 0 or len(part[0]) <= 0:
                continue

            cv2.imshow(
                "a",
                 code.skin_color_detection(
                      part
                 )
            )

            print(np.mean(part, axis=(0,1)))
            # print(part)

            cv2.imshow(f' {i}', part)

        prev_time = new_time
        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 255), 3)

        cv2.imshow("Hand Tracking", img)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

0 (487, 866)
0 (496, 871)
0 (490, 890)
0 (481, 895)
0 (484, 894)
0 (485, 892)
0 (485, 892)
0 (500, 867)
0 (508, 860)
0 (506, 861)
0 (491, 881)
0 (487, 885)
0 (487, 885)
0 (487, 883)
0 (573, 506)
0 (560, 352)
[54.2375 55.98   67.105 ]
0 (491, 251)
[78.825  76.01   81.8275]
0 (465, 242)
[77.4325 78.8625 86.8175]
0 (461, 249)
[73.2625 70.92   73.8625]
0 (459, 243)
[78.005 79.995 86.975]
0 (456, 241)
[103.08   81.815 111.93 ]
0 (454, 242)
[112.0125  79.2325 118.3225]
0 (454, 249)
[104.88    68.3075 100.9425]
0 (454, 251)
[103.1125  65.645   97.5575]
0 (454, 244)
[105.3525  68.0025 101.53  ]
0 (446, 243)
[ 98.6825  74.71   101.8675]
0 (442, 237)
[103.2225  82.915  114.3325]
0 (441, 224)
[107.0725  91.74   127.3275]
0 (444, 230)
[113.6025  81.0175 120.6525]
0 (445, 234)
[112.6     79.0025 118.6925]
0 (446, 235)
[114.17    83.24   125.5725]
0 (450, 240)
[112.4675  80.865  122.9375]
0 (450, 241)
[112.49    83.12   125.9625]
0 (453, 241)
[113.2625  84.3    129.0825]
0 (456, 241)
[114.93    85.8

In [5]:
cap.release()
cv2.destroyAllWindows()